This pairing combines a trend-following indicator with a momentum oscillator to provide confirmation of potential trend changes. 
Moving Average Convergence Divergence (MACD): Identifies momentum and potential trend changes through the crossover of its two moving average lines.
Usage: A bullish crossover (MACD line crosses above the signal line) can signal momentum is shifting to the upside.
Relative Strength Index (RSI): Confirms momentum and helps avoid false signals from the MACD.
Usage: An oversold RSI reading followed by a MACD bullish crossover provides more credible confirmation of a potential trend reversal. 

In [1]:
import pandas as pd
import requests
import numpy as np
from lightweight_charts import Chart
from stock_indicators import indicators, Quote
from datetime import datetime, timedelta
import asyncio
import nest_asyncio

nest_asyncio.apply()

In [2]:
# import yfinance as yf
# df = yf.download('SPY', start='2010-01-01', multi_level_index=False)
# df.reset_index(inplace=True)
# df.to_csv('SPY.csv', index=False)
df = pd.read_csv('SPY.csv')
rawdf = df.copy()
df['Date'] = pd.to_datetime(df['Date'])
df.head()

,Date,Close,High,Low,Open,Volume
0,2010-01-04,85.279205,85.324353,83.909682,84.556820,118944600
1,2010-01-05,85.504936,85.542563,84.917998,85.226520,111579900
2,2010-01-06,85.565147,85.775842,85.354452,85.422173,116074400
3,2010-01-07,85.926346,86.031693,85.166334,85.407129,131091100
4,2010-01-08,86.212257,86.249884,85.527499,85.700567,126402800


In [3]:
quotes = [
    Quote(d, o, h, l, c, v)
    for d, o, h, l, c, v in zip(
        df['Date'],
        df['Open'],
        df['High'],
        df['Low'],
        df['Close'],
        df['Volume']
    )
]


In [4]:
# Moving Average Convergence / Divergence (MACD)
df['macd'] = [r.macd for r in indicators.get_macd(quotes, 12, 26, 9)]
df['signal'] = [r.signal for r in indicators.get_macd(quotes, 12, 26, 9)]
df['histogram'] = [r.histogram for r in indicators.get_macd(quotes, 12, 26, 9)]
# df['bullishMACD'] = 0.0
df['MACD_signal'] = np.where(df['macd'] > df['signal'], 'long', 'short')
# df['crossover_MACD'] = df['bullishMACD'].diff()

# Calculate RSI
df['rsi'] = [r.rsi for r in indicators.get_rsi(quotes, 14)]
df['rsima6'] = df['rsi'].rolling(6).mean()
df['rsima14'] = df['rsi'].rolling(14).mean()
# df['rsi_bullish'] = 0.0
df['RSI_signal'] = np.where(df['rsi'] > df['rsima14'], 'long', 'short')
# df['crossover_rsi'] = df['rsi_bullish'].diff()


In [5]:
df = df.dropna().reset_index(drop=True)
df.head()


,Date,Close,High,Low,Open,Volume,macd,signal,histogram,MACD_signal,rsi,rsima6,rsima14,RSI_signal
0,2010-02-22,83.646309,83.962351,83.397987,83.939778,132346900,-0.677002,-1.432508,0.755506,long,55.167447,51.564492,44.322787,long
1,2010-02-23,82.630447,83.676402,82.412226,83.420560,207497000,-0.617656,-1.269538,0.651882,long,49.034546,52.562926,44.606893,long
2,2010-02-24,83.390480,83.525928,82.668094,82.878789,176350700,-0.503491,-1.116329,0.612838,long,53.224276,53.107540,45.322284,long
3,2010-02-25,83.277603,83.337803,81.975806,82.201548,259634700,-0.417312,-0.976525,0.559213,long,52.533553,53.216386,46.648929,long
4,2010-02-26,83.330246,83.616194,82.856182,83.352820,173589300,-0.340838,-0.849388,0.508550,long,52.840935,52.981833,47.928014,long


In [7]:
# Moving Average Convergence / Divergence (MACD) and RSI Combined Strategy
if __name__ == '__main__':
    
    chart = Chart(title="MACD + RSI Strategy", maximize=True, inner_height=0.7)
    chart.legend(visible=True, color_based_on_candle=True)

    # Set the main candlestick data for the chart
    chart.set(df)

    # Create MACD subchart
    macd_chart = chart.create_subchart(position='left', width=1.0, height=0.15, sync=True)
    macd_line = macd_chart.create_line('MACD', color="#2ee30f", width=1, price_line=False, price_label=False)
    macd_line.set(df[['Date', 'macd']])

    signal_line = macd_chart.create_line('Signal', color="#f36021", width=1, price_line=False, price_label=False)
    signal_line.set(df[['Date', 'signal']])

    histogram_bar = macd_chart.create_histogram('Histogram', color="#1341e4", price_line=False, price_label=False)
    histogram_bar.set(df[['Date', 'histogram']])

    # Create RSI subchart
    rsi_chart = chart.create_subchart(position='left', width=1.0, height=0.15, sync=True)
    rsi_line = rsi_chart.create_line('RSI', color="#e11d74", width=1, price_line=False, price_label=False)
    rsi_line.set(df[['Date', 'rsi']])
    
    # Add RSI MA lines
    # Create temporary DataFrames with renamed columns for the MA lines
    rsi_ma6_df = df[['Date', 'rsima6']].rename(columns={'rsima6': 'RSI MA(6)'})
    rsi_ma14_df = df[['Date', 'rsima14']].rename(columns={'rsima14': 'RSI MA(14)'})
    
    rsi_ma6 = rsi_chart.create_line('RSI MA(6)', color="#f7e01d", width=1, price_line=False, price_label=False)
    rsi_ma6.set(rsi_ma6_df)
    
    rsi_ma14 = rsi_chart.create_line('RSI MA(14)', color="#1df7eb", width=1, price_line=False, price_label=False)
    rsi_ma14.set(rsi_ma14_df)

    # Add overbought/oversold lines
    overbought_line = rsi_chart.create_line('Overbought', color="#ff0000", width=1, price_line=True, price_label=True)
    overbought_line.set(pd.DataFrame({'Date': df['Date'], 'Overbought': [70] * len(df)}))
    
    oversold_line = rsi_chart.create_line('Oversold', color="#00ff00", width=1, price_line=True, price_label=True)
    oversold_line.set(pd.DataFrame({'Date': df['Date'], 'Oversold': [30] * len(df)}))

    # Initialize a list to hold the markers
    markers = []

    # Iterate through the DataFrame to find trading signals
    for i in range(1, len(df)):
        macdSignal = df.iloc[i]['MACD_signal']
        p_macdSignal = df.iloc[i-1]['MACD_signal']
        rsiSignal = df.iloc[i]['RSI_signal']
        p_rsiSignal = df.iloc[i-1]['RSI_signal']
        current_time = df.iloc[i]['Date']

        # Buy signal: MACD crosses above signal line AND RSI is oversold (< 30)
        if (p_macdSignal == "short" and p_rsiSignal == "short") and (macdSignal == "long" and rsiSignal == "long"):
            markers.append({
                'time': current_time,
                'position': 'below', 
                'shape': 'arrow_up',
                'color': '#33de3d',
                'text': 'Buy'
            })
        
        # Sell signal: MACD crosses below signal line AND RSI is overbought (> 70)
        elif (p_macdSignal == "long" and p_rsiSignal == "long") and (macdSignal == "short" and rsiSignal == "short"):
            markers.append({
                'time': current_time,
                'position': 'above',
                'shape': 'arrow_down',
                'color': '#f485fb',
                'text': 'Sell'
            })

    # Add all markers at once
    if markers:
        chart.marker_list(markers)
    
chart.show(block=True)